In [1]:
from qmcpy.discrete_distribution import Sobol
from qmcpy.integrand._integrand import Integrand
from qmcpy.true_measure import BrownianMotion
from qmcpy.util import ParameterError
from numpy import *
import qmcpy as qp

In [59]:
class AsianOptionNew(Integrand):
    
    def __init__(self, sampler, volatility=0.5, start_price=30., strike_price=35.,
                 interest_rate=0.01, t_final=1, call_put='call', opt_type='amean'):
        """
        Args:
            sampler (DiscreteDistribution/TrueMeasure): A 
                discrete distribution from which to transform samples or a
                true measure by which to compose a transform
            volatility (float): sigma, the volatility of the asset
            start_price (float): S(0), the asset value at t=0
            strike_price (float): strike_price, the call/put offer
            interest_rate (float): r, the annual interest rate
            t_final (float): exercise time
            dim: number of observations in t_final
            call_put (string): 'call' or 'put'
            opt_type (string): 'amean' or 'gmean'
        """
        
        
        self.parameters = ['volatility', 'call_put', 'start_price', 'strike_price', 'interest_rate', 'opt_type']
        self.t_final = t_final
        self.true_measure = BrownianMotion(sampler,self.t_final)
        self.volatility = float(volatility)
        self.start_price = float(start_price)
        self.strike_price = float(strike_price)
        self.interest_rate = float(interest_rate)
        self.call_put = call_put.lower()
        if self.call_put not in ['call','put']:
            raise ParameterError("call_put must be either 'call' or 'put'")
        self.opt_type = opt_type.lower()
        if self.opt_type not in ['amean', 'gmean']:
            raise ParameterError("opt_type must either 'amean' or 'gmean'")
        # single level problem
        self.dimensions = self.true_measure.d
        super(AsianOptionNew,self).__init__()    
        
        
    def g(self, t):
        """ See abstract method. """
        
        dimension = float(self.dimensions)
        self.stock_path= self.start_price * exp(
            (self.interest_rate - self.volatility ** 2 / 2.) *
            self.true_measure.time_vec + self.volatility * t)
        for xx,yy in zip(*where(self.stock_path<0)): # if stock becomes <=0, 0 out rest of path
            self.stock_path[xx,yy:] = 0
            
        if self.opt_type == 'amean':
            y_raw = (self.stock_path.sum(1) + self.start_price / 2.0 - \
                   self.stock_path[:, -1] / 2.0) / float(dimension)
        else:
            y_raw = exp((log(self.stock_path).sum(1) + log(self.start_price) / 2.0 - \
                   log(self.stock_path[:, -1]) / 2.0) / float(dimension))                 

        
        if self.call_put == 'call':
            y_raw = maximum(y_raw - self.strike_price, 0)
        else: # put
            y_raw = maximum(self.strike_price - y_raw, 0)  
            
        y = y_raw * exp(-self.interest_rate * self.t_final)
        return y

In [60]:
al = AsianOptionNew(sampler = qp.IIDStdUniform(dimension=6),
                          volatility = 0.19654,
                          call_put = 'call',
                          start_price=537.36,
                          strike_price =600,
                          interest_rate=0.0050238,
                          opt_type = 'amean',
                          t_final = 1/2)

In [61]:
sc = qp.CubMCG(al,abs_tol=.05)
solution, data = sc.integrate()

In [62]:
solution

1.881976965561897

In [63]:
data

Solution: 1.8820         
AsianOptionNew (Integrand Object)
    volatility      0.197
    call_put        call
    start_price     537.360
    strike_price    600
    interest_rate   0.005
    opt_type        amean
IIDStdUniform (DiscreteDistribution Object)
    d               6
    seed            594737
    mimics          StdUniform
BrownianMotion (TrueMeasure Object)
    time_vec        [0.083 0.167 0.25  0.333 0.417 0.5  ]
    drift           0
    mean            [0. 0. 0. 0. 0. 0.]
    covariance      [[0.083 0.083 0.083 0.083 0.083 0.083]
                    [0.083 0.167 0.167 0.167 0.167 0.167]
                    [0.083 0.167 0.25  0.25  0.25  0.25 ]
                    [0.083 0.167 0.25  0.333 0.333 0.333]
                    [0.083 0.167 0.25  0.333 0.417 0.417]
                    [0.083 0.167 0.25  0.333 0.417 0.5  ]]
    decomp_type     pca
CubMCG (StoppingCriterion Object)
    inflate         1.200
    alpha           0.010
    abs_tol         0.050
    rel_tol        

In [49]:
agmo = AsianOptionNew(sampler = qp.IIDStdUniform(dimension=6),
                          volatility=0.19654,
                          call_put='call',
                          start_price=537.36,
                          strike_price=600,
                          interest_rate=0.0050238,
                          opt_type='gmean',
                          t_final=1/2)

In [50]:
agmo

AsianOptionNew (Integrand Object)
    volatility      0.197
    call_put        call
    start_price     537.360
    strike_price    600
    interest_rate   0.005
    opt_type        gmean

In [51]:
agmo_solution, agmo_data = qp.CubMCG(agmo,abs_tol=.05).integrate()
agmo_solution

1.7127821098462452

In [52]:
agmo_data

Solution: 1.7128         
AsianOptionNew (Integrand Object)
    volatility      0.197
    call_put        call
    start_price     537.360
    strike_price    600
    interest_rate   0.005
    opt_type        gmean
IIDStdUniform (DiscreteDistribution Object)
    d               6
    seed            716714
    mimics          StdUniform
BrownianMotion (TrueMeasure Object)
    time_vec        [0.083 0.167 0.25  0.333 0.417 0.5  ]
    drift           0
    mean            [0. 0. 0. 0. 0. 0.]
    covariance      [[0.083 0.083 0.083 0.083 0.083 0.083]
                    [0.083 0.167 0.167 0.167 0.167 0.167]
                    [0.083 0.167 0.25  0.25  0.25  0.25 ]
                    [0.083 0.167 0.25  0.333 0.333 0.333]
                    [0.083 0.167 0.25  0.333 0.417 0.417]
                    [0.083 0.167 0.25  0.333 0.417 0.5  ]]
    decomp_type     pca
CubMCG (StoppingCriterion Object)
    inflate         1.200
    alpha           0.010
    abs_tol         0.050
    rel_tol        

In [67]:
class LookbackOption(Integrand):
    
    def __init__(self, sampler, volatility=0.5, start_price=30.,
                 interest_rate=0.01, t_final=1, call_put='call'):
        """
        Args:
            sampler (DiscreteDistribution/TrueMeasure): A 
                discrete distribution from which to transform samples or a
                true measure by which to compose a transform
            volatility (float): sigma, the volatility of the asset
            start_price (float): S(0), the asset value at t=0
            interest_rate (float): r, the annual interest rate
            t_final (float): exercise time
            call_put (string): 'call' or 'put'
            dim: number of observations in t_final
        """
        
        
        self.parameters = ['volatility', 'call_put', 'start_price', 'interest_rate']
        self.t_final = t_final
        self.true_measure = BrownianMotion(sampler,self.t_final)
        self.volatility = float(volatility)
        self.start_price = float(start_price)
        self.interest_rate = float(interest_rate)
        self.call_put = call_put.lower()
        if self.call_put not in ['call','put']:
            raise ParameterError("call_put must be either 'call' or 'put'")
       # single level problem
        self.dimensions = self.true_measure.d
        super(LookbackOption,self).__init__()    
        
        
    def g(self, t):
        """ See abstract method. """
        
        dimension = float(self.dimensions)
        self.stock_path= self.start_price * exp(
            (self.interest_rate - self.volatility ** 2 / 2.) *
            self.true_measure.time_vec + self.volatility * t)
        for xx,yy in zip(*where(self.stock_path<0)): # if stock becomes <=0, 0 out rest of path
            self.stock_path[xx,yy:] = 0
            
        if self.call_put == 'put':
            y_raw = maximum(maximum(self.stock_path.max(1),self.start_price) - self.stock_path[:,-1],0)
        else:
            y_raw = maximum(self.stock_path[:,-1]-minimum(self.stock_path.min(1),self.start_price),0) 

            
        y = y_raw * exp(-self.interest_rate * self.t_final)
        return y

In [68]:
lb = LookbackOption(sampler = qp.IIDStdUniform(dimension=6),
                          volatility=0.5,
                          call_put='call',
                          start_price=120,
                          interest_rate=0.02,
                          t_final = 1/2)

In [69]:
sc = qp.CubMCG(lb,abs_tol=.05)
solution, data = sc.integrate()
solution

24.04790065156598

In [70]:
solution

24.04790065156598

In [71]:
data

Solution: 24.0479        
LookbackOption (Integrand Object)
    volatility      2^(-1)
    call_put        call
    start_price     120
    interest_rate   0.020
IIDStdUniform (DiscreteDistribution Object)
    d               6
    seed            748703
    mimics          StdUniform
BrownianMotion (TrueMeasure Object)
    time_vec        [0.083 0.167 0.25  0.333 0.417 0.5  ]
    drift           0
    mean            [0. 0. 0. 0. 0. 0.]
    covariance      [[0.083 0.083 0.083 0.083 0.083 0.083]
                    [0.083 0.167 0.167 0.167 0.167 0.167]
                    [0.083 0.167 0.25  0.25  0.25  0.25 ]
                    [0.083 0.167 0.25  0.333 0.333 0.333]
                    [0.083 0.167 0.25  0.333 0.417 0.417]
                    [0.083 0.167 0.25  0.333 0.417 0.5  ]]
    decomp_type     pca
CubMCG (StoppingCriterion Object)
    inflate         1.200
    alpha           0.010
    abs_tol         0.050
    rel_tol         0
    n_init          2^(10)
    n_max           100

In [72]:
lb = LookbackOption(sampler = qp.IIDStdUniform(dimension=6),
                          volatility=0.5,
                          call_put='put',
                          start_price=120,
                          interest_rate=0.02,
                          t_final = 1/2)

In [73]:
sc = qp.CubMCG(lb,abs_tol=.05)
solution, data = sc.integrate()
solution

25.97022135846145

In [74]:
data

Solution: 25.9702        
LookbackOption (Integrand Object)
    volatility      2^(-1)
    call_put        put
    start_price     120
    interest_rate   0.020
IIDStdUniform (DiscreteDistribution Object)
    d               6
    seed            705483
    mimics          StdUniform
BrownianMotion (TrueMeasure Object)
    time_vec        [0.083 0.167 0.25  0.333 0.417 0.5  ]
    drift           0
    mean            [0. 0. 0. 0. 0. 0.]
    covariance      [[0.083 0.083 0.083 0.083 0.083 0.083]
                    [0.083 0.167 0.167 0.167 0.167 0.167]
                    [0.083 0.167 0.25  0.25  0.25  0.25 ]
                    [0.083 0.167 0.25  0.333 0.333 0.333]
                    [0.083 0.167 0.25  0.333 0.417 0.417]
                    [0.083 0.167 0.25  0.333 0.417 0.5  ]]
    decomp_type     pca
CubMCG (StoppingCriterion Object)
    inflate         1.200
    alpha           0.010
    abs_tol         0.050
    rel_tol         0
    n_init          2^(10)
    n_max           1000

In [149]:
class BarrierOption(Integrand):
    
    def __init__(self, sampler, volatility=0.5, start_price=30., barrier_price = 35., strike_price = 30.,
                 interest_rate=0.01, t_final=1, call_put='call',barrier_type = 'upin'):
        """
        Args:
            sampler (DiscreteDistribution/TrueMeasure): A 
                discrete distribution from which to transform samples or a
                true measure by which to compose a transform
            volatility (float): sigma, the volatility of the asset
            start_price (float): S(0), the asset value at t=0
            strike_price (float): strike_price, the call/put offer
            interest_rate (float): r, the annual interest rate
            t_final (float): exercise time
            dim: number of observations in t_final
            call_put (string): 'call' or 'put'
            barrier_type (string): 'upin', 'upout', 'downin' or 'downout'
        """
        
        
        self.parameters = ['volatility', 'call_put', 'start_price', 'interest_rate',
                           'strike_price', 'barrier_price', 'barrier_type']
        self.t_final = t_final
        self.true_measure = BrownianMotion(sampler,self.t_final)
        self.volatility = float(volatility)
        self.start_price = float(start_price)
        self.strike_price = float(strike_price)
        self.barrier_price = float(barrier_price)
        self.interest_rate = float(interest_rate)
        self.call_put = call_put.lower()
        if self.call_put not in ['call','put']:
            raise ParameterError("call_put must be either 'call' or 'put'")
       # single level problem
        self.barrier_type = barrier_type.lower()
        if self.barrier_type not in ['upin','upout','downin','downout']:
            raise ParameterError("barrier_type must be in 'upin', 'upout', 'downin', 'downout'")
        self.dimensions = self.true_measure.d
        super(BarrierOption,self).__init__()    
        
        
    def g(self, t):
        """ See abstract method. """
        
        dimension = float(self.dimensions)
        self.stock_path= self.start_price * exp(
            (self.interest_rate - self.volatility ** 2 / 2.) *
            self.true_measure.time_vec + self.volatility * t)
        for xx,yy in zip(*where(self.stock_path<0)): # if stock becomes <=0, 0 out rest of path
            self.stock_path[xx,yy:] = 0
        
        yraw = zeros(self.stock_path.shape[0])
        if self.barrier_type == 'upin' and self.start_price <  self.barrier_price:
            temp = self.stock_path.max(1) > self.barrier_price      
        elif self.barrier_type == 'downin'and self.start_price > self.barrier_price:
            temp = self.stock_path.min(1) < self.barrier_price
        elif self.barrier_type == 'upout' and self.start_price < self.barrier_price:
            temp = self.stock_path.max(1) < self.barrier_price
        elif self.barrier_type == 'downout' and self.start_price > self.barrier_price:
            temp = self.stock_path.min(1) > self.barrier_price
        yraw[temp] = self.stock_path[temp, -1]
        
        if self.call_put == 'call':          
            yraw = maximum(yraw - self.strike_price, 0)
        else:
            yraw = maximum(self.strike_price - yraw, 0)          
        
        y = yraw * exp(-self.interest_rate * self.t_final)
        return y

In [153]:
upin_option = BarrierOption(sampler = qp.IIDStdUniform(dimension=13),
                          volatility=0.5,
                          call_put='call',
                          start_price=120,
                          strike_price=120,
                          interest_rate=0.02,
                          barrier_type='upout',
                          barrier_price=150,
                          t_final=1/4)

In [154]:
upin_option

BarrierOption (Integrand Object)
    volatility      2^(-1)
    call_put        call
    start_price     120
    interest_rate   0.020
    strike_price    120
    barrier_price   150
    barrier_type    upout

In [155]:
sc = qp.CubMCG(upin_option,abs_tol=.05)
solution, data = sc.integrate()
solution

2.408599620928891

In [139]:
bm = BrownianMotion(Sobol(3),t_final=2,drift=2)
samp = bm.gen_samples(4)
samp

array([[1.118, 4.292, 5.584],
       [0.898, 1.676, 2.822],
       [2.364, 3.155, 4.691],
       [0.877, 2.404, 3.311]])

In [123]:
where = samp.max(1)>3

In [124]:
where

array([ True,  True,  True, False])

In [125]:
yraw = zeros(samp.shape[1])
yraw[where] = samp[where,-1]

In [126]:
yraw

array([6.795, 4.084, 5.2  , 0.   ])

In [140]:
samp.shape[1]

3